<a href="https://colab.research.google.com/github/callor/Callor-AI-2025/blob/master/Llama_%ED%8C%8C%EC%9D%B8%ED%8A%9C%EB%8B%9D_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
from datasets import load_dataset, Dataset
import torch

from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# 모델과 토크나이저 로드
base_model = "meta-llama/Meta-Llama-3-8B"

tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True, token='YOUR_HF_TOKEN')
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    device_map="auto",
    token='YOUR_HF_TOKEN'  # Pass the token here as well
)


tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    device_map="auto",
)

# LoRA 설정
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

# 예시용 데이터셋
# 실제로는 instruction-output 형태로 준비된 JSONL 파일을 Dataset으로 로드해야 함
example_data = {
    "instruction": ["Java로 파일을 읽는 코드를 작성해줘."],
    "output": [
        "import java.io.*;\npublic class ReadFile {\n public static void main(String[] args) throws IOException {\n BufferedReader reader = new BufferedReader(new FileReader(\"input.txt\"));\n String line;\n while ((line = reader.readLine()) != null) {\n System.out.println(line);\n }\n reader.close();\n }\n}"
    ]
}

In [ ]:
def format_prompt(example):
    return f"<s>[INST] {example['instruction']} [/INST] {example['output']} </s>"

# 데이터 전처리
raw_dataset = Dataset.from_dict(example_data)
raw_dataset = raw_dataset.map(lambda x: {"text": format_prompt(x)})

In [ ]:
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = raw_dataset.map(tokenize_function, batched=True)

In [ ]:
# 학습 인자 설정
training_args = TrainingArguments(
    output_dir="./llama3-java-lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    remove_unused_columns=False,
)

In [ ]:
# 데이터 로더용 데이터 수집기
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Trainer 실행
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

# 저장
trainer.save_model("./llama3-java-lora")
tokenizer.save_pretrained("./llama3-java-lora")